# `graphmix` examples

This notebook provides several examples to demonstrate the usage of
the `graphmix` package.

## Install `graphmix`


## Import `graphmix` into your code

Graphmix uses a chemical registry to store chemical data. The chemical registry
is a SQLite database that stores chemical data from PubChem. By default, the
database is located at `~/.graphmix/chemicals.db`. If the database does not exist, 
it will be created when you first initialize a `ChemicalRegistry` object. You can 
optionally pass a database path to the `ChemicalRegistry` constructor to specify a 
different location for the database.


In [4]:
%load_ext autoreload
%autoreload 2

In [6]:
import graphmix as gm

chem_reg = gm.ChemicalRegistry()


## Examples

### Example 1: Get chemical data from PubChem

In [5]:
etoh = chem_reg.Chemical('Ethanol')

print(etoh)

name='Ethanol' formula='C2H6O' smiles='CCO' molar_mass=<Quantity(46.07, 'gram / mole')> id=None
